In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class SegmentTree:
  def __init__(self, L, default, f):
    self._def = default
    self._f = f
    self._len = len(L)
    self._size = _size = 1 << (self._len - 1).bit_length()

    self.L = [default] * (2 * _size)
    self.L[_size:_size + self._len] = L
    for i in reversed(range(_size)):
      self.L[i] = f(self.L[i + i], self.L[i + i + 1])

  def __delitem__(self, idx):
    self[idx] = self._def

  def __getitem__(self, idx):
    return self.L[idx + self._size]

  def __setitem__(self, idx, value):
    idx += self._size
    self.L[idx] = value
    idx >>= 1
    while idx:
      self.L[idx] = self._f(self.L[2 * idx], self.L[2 * idx + 1])
      idx >>= 1

  def __len__(self):
    return self._len

  def query(self, s, e):
    s += self._size
    e += self._size

    l = r = self._def
    while s < e: 
      if s & 1:
        l = self._f(l, self.L[s])
        s += 1
      if e & 1:
        e -= 1
        r = self._f(self.L[e], r)
      s >>= 1
      e >>= 1

    return self._f(l, r)

from operator import add
def sol() :
  N, M = map(int, input().split())
  L = list(map(int, input().split()))

  st = SegmentTree([1] * N + [0] * M, 0, add)
  pos = [*reversed(range(N))] #i번째 DVD가 있는 인덱스
  pt = 0 #세그먼트상 DVD가 들어갈 인덱스
  ans = []
  for l in L :
    l -= 1
    ans.append(st.query(pos[l]+1, N+M)) #pos[l]보다 뒤에 있는 1의 개수 = 쌓인 DVD의 개수
    #위치 정보를 올바르게 갱신한다. 원본 자리에 있던 DVD는 빼고, 새로운 자리에 DVD를 넣는다.
    st[pos[l]] = 0 
    st[N + pt] = 1
    pos[l] = N + pt 

    pt += 1
  sys.stdout.write(' '.join(map(str, ans)) + '\n')

for _ in range(int(input())) :
  sol()

### 풀이
- N + M 길이의 세그먼트 트리를 초기화한다. 앞 N개의 값은 1로, DVD가 그 위치에 존재한다는 뜻이고, 그 뒤는 0이다. 쿼리 하나마다 1씩 늘어난다.
- i번째 DVD가 속한 위치 정보를 저장하는 list를 초기화한다. 1번 DVD가 맨 위에있으니, 1번 위치에 존재할 DVD는 N, 즉 reversed(range(N)) 이다.
- 쿼리를 처리하면서 위치 정보를 갱신한다. 구현을 살펴보자.